# 4. ANALYTICS

In [1]:
import pandas as pd
import pandasql as psql
from xgboost import XGBRegressor
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import cross_val_score
import pandas as pd
from joblib import dump
import matplotlib.pyplot as plt
import numpy as np
import time

In [2]:
path_structured = 'C:\\Users\\anton\\OneDrive\\Documentos\\GitHub\\UniversityHack2024\\data\\03_structured\\'
path_analytics = 'C:\\Users\\anton\\OneDrive\\Documentos\\GitHub\\UniversityHack2024\\data\\04_analytics\\'

## 4.1 TRAIN

In [3]:
path_materias_03 = path_structured + 'materias_primas_total.xlsx'
path_preinoculo_03 = path_structured + 'preinoculo_total.xlsx'
path_inoculo_03 = path_structured + 'inoculo_total.xlsx'
path_cultivo_03 = path_structured + 'cultivo_final_total.xlsx'
path_cent_03 = path_structured + 'centrifugacion_total.xlsx'


# Leer el archivo Excel y guardar el contenido en un DataFrame
df_materiales_03 = pd.read_excel(path_materias_03)
df_preinoculo_03 = pd.read_excel(path_preinoculo_03)
df_inoculo_03 = pd.read_excel(path_inoculo_03)
df_cultivo_03 = pd.read_excel(path_cultivo_03)
df_cent_03 = pd.read_excel(path_cent_03)

In [8]:
consulta = ''' 
    SELECT
    *
    FROM 
        df_cultivo_03 cu

    LEFT JOIN 
        df_materiales_03 m
    ON 
        CAST(m.Lote AS TEXT) = CAST(cu.LOTE AS TEXT)

    LEFT JOIN 
        df_preinoculo_03 pre
    ON
        pre.LOTE = CAST(COALESCE(cu.lote_abuelo, cu.`LOTE parental`, cu.LOTE) AS TEXT)

    LEFT JOIN
        df_inoculo_03 ino
    ON
        ino.LOTE = CAST(COALESCE(cu.lote_abuelo, cu.`LOTE parental`, cu.LOTE) AS TEXT)

    LEFT JOIN
        df_cent_03 cent
    ON
        CAST(cent.Lote AS TEXT) = CAST(cu.LOTE AS TEXT)

'''
# Ejecutamos la consulta y generamos un dataframe
df_train = psql.sqldf(consulta, locals())

,LOTE,Orden en el encadenado,LOTE parental,ID Bioreactor,Volumen de inóculo utilizado,Turbidez inicio cultivo,Turbidez fin cultivo,Viabilidad final cultivo,ID Centrífuga,Centrifugación 1 turbidez,...,avg_descargas_totales_2,min_descargas_totales_2,max_descargas_totales_2,avg_descargas_parciales_2,min_descargas_parciales_2,max_descargas_parciales_2,Turbidez_2_100,Turbidez_2_200,Turbidez_2_300,Turbidez_2_400
0,23019,1,NaN,14615,82.40,17.28,91.20,184000000.0,17825,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.80,20.80,21.60,21.52
1,23020,1,NaN,14616,80.40,18.80,91.20,181600000.0,14246,NaN,...,NaN,NaN,NaN,6650.426390,6518.431966,6677.551716,12.80,13.04,12.88,12.88
2,23021,1,NaN,13170,66.40,16.16,86.40,248000000.0,17825,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.48,20.48,17.84,20.24
3,23022,1,NaN,14614,85.60,18.48,83.20,229600000.0,12912,NaN,...,NaN,NaN,NaN,6678.741666,6677.179237,6681.402561,10.08,10.00,11.04,9.44
4,23023,1,NaN,14615,77.60,17.12,74.40,132800000.0,17825,26.56,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,21.04,20.96,20.80,20.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,24049,1,NaN,14617,83.60,18.88,72.64,164800000.0,12912,30.56,...,4.000000,4.000000,4.000000,0.000000,0.000000,0.000000,17.28,NaN,NaN,16.72
151,24050,1,NaN,14614,84.16,17.76,67.60,152000000.0,6379,29.44,...,NaN,NaN,NaN,NaN,NaN,NaN,26.72,NaN,NaN,26.56
152,24051,1,NaN,13169,84.16,17.76,80.80,160800000.0,12912,33.44,...,1.592191,1.542150,1.652972,6672.284397,6670.650781,6675.256872,20.08,NaN,NaN,18.56
153,24052,2,24050.0,14614,86.40,17.28,69.04,148000000.0,14246,23.68,...,1.584164,1.557465,1.622271,6680.530915,6678.185938,6684.106250,18.24,NaN,NaN,18.16


In [24]:
# Guardamos el resultado en un archivo Excel
df_train.to_excel(f'{path_analytics}train.xlsx', index=False)

## 4.2 TEST

In [4]:
path_cultivo_test_03 = path_structured + 'cultivo_final_test_total.xlsx'

# Leer el archivo Excel y guardar el contenido en un DataFrame
df_cultivo_test_03 = pd.read_excel(path_cultivo_test_03)

In [7]:
consulta = ''' 
    SELECT
    *
    FROM 
        df_cultivo_test_03 cu

    LEFT JOIN 
        df_materiales_03 m
    ON 
        CAST(m.Lote AS TEXT) = CAST(cu.LOTE AS TEXT)

    LEFT JOIN 
        df_preinoculo_03 pre
    ON
        pre.LOTE = CAST(COALESCE(cu.lote_abuelo, cu.LOTE_parental, cu.LOTE) AS TEXT)

    LEFT JOIN
        df_inoculo_03 ino
    ON
        ino.LOTE = CAST(COALESCE(cu.lote_abuelo, cu.LOTE_parental, cu.LOTE) AS TEXT)

    LEFT JOIN
        df_cent_03 cent
    ON
        CAST(cent.Lote AS TEXT) = CAST(cu.LOTE AS TEXT)

'''
# Ejecutamos la consulta y generamos un dataframe
df_test = psql.sqldf(consulta, locals())

,LOTE,Orden,LOTE_parental,ID_Bioreactor,horas_totales_cultivofinal,Volumen_inoculo_utilizado,Turbidez_inicio_cultivo,Turbidez_fin_cultivo,Viabilidad_final_cultivo,ID_Centrifuga,...,avg_descargas_totales_2,min_descargas_totales_2,max_descargas_totales_2,avg_descargas_parciales_2,min_descargas_parciales_2,max_descargas_parciales_2,Turbidez_2_100,Turbidez_2_200,Turbidez_2_300,Turbidez_2_400
0,24054,1,NaN,14616,47.266667,81.60,15.44,85.60,184800000,14246,...,1.544979,0.598974,2.271431,4264.154146,0.0,6682.760156,23.84,None,None,24.08
1,24055,1,NaN,14614,48.200000,NaN,14.32,73.68,175200000,12912,...,2.105049,1.552790,4.000000,4562.504478,0.0,6681.626022,24.08,None,None,22.24
2,24056,1,NaN,14615,47.950000,NaN,14.56,82.40,168000000,14246,...,1.297109,0.530158,1.624594,3678.405267,0.0,6683.029688,29.28,None,None,28.48
3,24057,1,NaN,13170,47.483333,82.40,17.76,78.96,180800000,12912,...,2.027010,1.554129,4.000000,4423.573467,0.0,6677.109375,25.20,None,None,25.44
4,24058,2,24055.0,14614,43.766667,87.20,18.00,82.40,144800000,12912,...,2.271915,1.465423,4.000000,4238.367548,0.0,6674.658186,20.16,None,None,20.56
5,24059,2,24056.0,14615,43.916667,87.20,18.08,78.40,166400000,14246,...,1.549943,0.659546,2.180878,3897.657719,0.0,6679.312271,23.20,None,None,24.08
6,24060,1,NaN,13169,47.883333,82.40,25.92,76.32,161600000,14246,...,1.421358,0.657117,2.223965,4531.724657,0.0,6681.577284,17.92,None,None,NaN
7,24061,1,NaN,14617,47.800000,81.60,28.56,68.96,148800000,12912,...,2.019146,1.507888,4.000000,4421.635188,0.0,6672.463175,15.44,None,None,14.00
8,24062,3,24058.0,14614,44.000000,87.20,18.88,80.00,164800000,12912,...,2.400370,1.553567,4.000000,4323.481685,0.0,6672.032489,24.64,None,None,NaN
9,24063,1,NaN,13170,48.650000,NaN,15.52,68.72,180800000,12912,...,2.131712,1.561314,4.000000,3160.092575,0.0,6670.124561,19.04,None,None,17.52


In [9]:
# Guardamos el resultado en un archivo Excel
df_test.to_excel(f'{path_analytics}test.xlsx', index=False)

## 4.3 PREDICTIONS

In [ ]:
train = pd.read_excel(f'{path_analytics}train.xlsx') #este train es de la carpeta model_input
y = train.PRODUCTO_1
X = train.drop('PRODUCTO_1', axis=1)

In [ ]:
#probamos con diversas combinaciones de hiperparametros para buscar los mejores mediante cross validation
#con los hiperparametros q mejores resultados obtengamos, reentrenaremos el modelo con todos los datos

metrics = {}
for n_estimators in [5, 10, 25, 50, 100, 250, 500]:
    for max_depth in [2, 3, 5, 7]:
        for eta in [0.05, 0.1, 0.2, 0.5]:
            model = XGBRegressor(n_estimators=n_estimators, max_depth=max_depth, learning_rate=eta)
            rmse_scores = - cross_val_score(model, X, y, scoring='neg_root_mean_squared_error')

            metrics[f'xgb_{n_estimators}_{max_depth}_{eta}'] = {
                'RMSE_1': rmse_scores[0],
                'RMSE_2': rmse_scores[1],
                'RMSE_3': rmse_scores[2],
                'RMSE_4': rmse_scores[3],
                'RMSE_5': rmse_scores[4],
                'MEAN_RMSE': rmse_scores.mean()
            }

metrics_xgb = pd.DataFrame.from_dict(metrics, orient='index',columns=['RMSE_1', 'RMSE_2', 'RMSE_3', 'RMSE_4', 'RMSE_5', 'MEAN_RMSE'])
metrics_xgb.sort_values(by='MEAN_RMSE') 

In [ ]:
#reentrenamos el modelo con los mejores hiperparametros
time1 = time.time()
xgb = XGBRegressor(n_estimators=250, max_depth=3, learning_rate=0.05, random_state=73).fit(X, y)
time2 = time.time() - time1
preds = xgb.predict(X)
rmse = root_mean_squared_error(y, preds)
print('Tiempo de ejecución: '+str(time2)+' segundos. \nRMSE sobre el train: '+str(rmse))

In [ ]:
dump(xgb, 'C:\\Users\\anton\\OneDrive\\Documentos\\GitHub\\UniversityHack2024\\01_Data\\06_model_output\\models\\xgboost.joblib')

In [ ]:
#plot para observar cómo se comportan las estimaciones sobre los valores reales (ejecútalo para que se vea)
plt.figure(figsize=(10, 8))
plt.scatter(y, preds)
plt.plot([y.min(), y.max()], [y.min(), y.max()], '--r', lw=2)
plt.xlabel('Valores Reales')
plt.ylabel('Predicciones')
plt.title('Predicciones vs valores reales XGBoost')
plt.show()

In [ ]:
#leemos el dataset de test
test = pd.read_excel('05_model_input/test.xlsx')

In [ ]:
#generamos las predicciones para el dataset de respuesta
pred = xgb.predict(test)
pred